Function calling in OpenAI

In [1]:
import os
import openai
from dotenv import load_dotenv

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [7]:
openai.api_key = OPENAI_API_KEY

In [3]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API

import requests
def get_current_weather(location):
    """Get the current weather in a given location"""

    url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"

    querystring = {"text":location}

    headers = {
      'x-rapidapi-key': "156877d1e1msh6806b57dfec44b9p1bd2c8jsn796f2ff15790",
      'x-rapidapi-host': "ai-weather-by-meteosource.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    print(response.json())
  
    return response.json()

In [10]:
response = get_current_weather("Washington DC")

[{'name': 'Washington D.C.', 'place_id': 'washington', 'adm_area1': 'District of Columbia', 'adm_area2': 'Washington', 'country': 'United States of America', 'lat': '38.89511N', 'lon': '77.03637W', 'timezone': 'America/New_York', 'type': 'settlement'}, {'name': 'District of Columbia', 'place_id': 'district-of-columbia', 'adm_area1': 'District of Columbia', 'adm_area2': None, 'country': 'United States of America', 'lat': '38.91706N', 'lon': '77.00025W', 'timezone': 'America/New_York', 'type': 'administrative_area'}, {'name': 'Fairfield Inn & Suites Washington. Dc Downtown', 'place_id': 'fairfield-inn-suites-washington-dc-downtown', 'adm_area1': 'District of Columbia', 'adm_area2': 'Washington', 'country': 'United States of America', 'lat': '38.89981N', 'lon': '77.01896W', 'timezone': 'America/New_York', 'type': 'heliport'}, {'name': 'District of Columbia General Hospital ACCC Heliport', 'place_id': 'district-of-columbia-general-hospital-accc-heliport', 'adm_area1': 'District of Columbia

In [11]:
response

[{'name': 'Washington D.C.',
  'place_id': 'washington',
  'adm_area1': 'District of Columbia',
  'adm_area2': 'Washington',
  'country': 'United States of America',
  'lat': '38.89511N',
  'lon': '77.03637W',
  'timezone': 'America/New_York',
  'type': 'settlement'},
 {'name': 'District of Columbia',
  'place_id': 'district-of-columbia',
  'adm_area1': 'District of Columbia',
  'adm_area2': None,
  'country': 'United States of America',
  'lat': '38.91706N',
  'lon': '77.00025W',
  'timezone': 'America/New_York',
  'type': 'administrative_area'},
 {'name': 'Fairfield Inn & Suites Washington. Dc Downtown',
  'place_id': 'fairfield-inn-suites-washington-dc-downtown',
  'adm_area1': 'District of Columbia',
  'adm_area2': 'Washington',
  'country': 'United States of America',
  'lat': '38.89981N',
  'lon': '77.01896W',
  'timezone': 'America/New_York',
  'type': 'heliport'},
 {'name': 'District of Columbia General Hospital ACCC Heliport',
  'place_id': 'district-of-columbia-general-hospit

In [ ]:
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    
                },
                "required": ["location"],
            },
        }
    ]

In [27]:
user_message="What is the temperature in Washington DC"
messages=[]
messages.append({"role": "user", "content": user_message})

completion=openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
    
)

In [28]:
completion

<OpenAIObject chat.completion id=chatcmpl-BJ5KynB2e3tNDk9acjT7kppDIgcI9 at 0x1175c85e0> JSON: {
  "id": "chatcmpl-BJ5KynB2e3tNDk9acjT7kppDIgcI9",
  "object": "chat.completion",
  "created": 1743888448,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\"location\":\"Washington DC\"}"
        },
        "refusal": null,
        "annotations": []
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 70,
    "completion_tokens": 17,
    "total_tokens": 87,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "service_tier": "default

In [29]:
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"location\":\"Washington DC\"}"
  },
  "refusal": null,
  "annotations": []
}


In [20]:
response=completion.choices[0].message

In [30]:
response

<OpenAIObject at 0x1175b67f0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"location\":\"Washington, DC\"}"
  },
  "refusal": null,
  "annotations": []
}

In [22]:
function_name=response['function_call']['name']
print(function_name)

get_current_weather


In [23]:
import json
location=eval(response['function_call']['arguments'])['location']
print(location)

Washington, DC


In [24]:
messages.append(response)  # extend conversation with assistant's reply
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": location,
    }
)

In [25]:
# extend conversation with function response
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)  # get a new response from GPT where it can see the function response


In [26]:
print(second_response.choices[0].message)

{
  "role": "assistant",
  "content": "The current temperature in Washington, DC is 75\u00b0F.",
  "refusal": null,
  "annotations": []
}
